In [1]:
import numpy as np

In [2]:
totals = np.genfromtxt('data/clean.csv', delimiter=',')[1:]  # drop header
totals

array([[  1.,   7.,  16., ...,   4.,   3.,  15.],
       [  2.,   5.,   8., ...,   1.,   1.,   5.],
       [  3.,   8.,  16., ...,  19.,  11.,   7.],
       ...,
       [398.,   3.,   0., ...,   7.,   3.,   2.],
       [399.,   4.,   1., ...,   2.,   0.,   3.],
       [400.,   3.,   2., ...,   3.,   6.,   5.]])

In [3]:
votes = totals[:, 1:]
votes

array([[ 7., 16., 10., ...,  4.,  3., 15.],
       [ 5.,  8., 20., ...,  1.,  1.,  5.],
       [ 8., 16.,  6., ..., 19., 11.,  7.],
       ...,
       [ 3.,  0.,  0., ...,  7.,  3.,  2.],
       [ 4.,  1.,  7., ...,  2.,  0.,  3.],
       [ 3.,  2.,  1., ...,  3.,  6.,  5.]])

In [4]:
num_classes = votes.shape[1]
num_classes

9

In [5]:
max_cats = votes.argmax(axis=1)
assert max_cats.shape[0] == votes.shape[0]
max_cats

array([4, 4, 6, 4, 3, 4, 6, 4, 4, 6, 3, 6, 3, 4, 3, 5, 4, 6, 4, 4, 4, 2,
       6, 6, 1, 6, 5, 6, 4, 6, 3, 4, 4, 6, 4, 4, 2, 4, 6, 6, 1, 4, 6, 4,
       4, 4, 8, 1, 8, 6, 3, 6, 7, 6, 4, 4, 1, 5, 6, 3, 6, 6, 4, 6, 6, 7,
       4, 4, 2, 3, 6, 1, 3, 7, 7, 7, 4, 6, 6, 6, 6, 4, 3, 1, 6, 7, 4, 6,
       4, 6, 8, 4, 4, 1, 6, 4, 1, 4, 4, 5, 3, 3, 6, 3, 8, 3, 4, 3, 5, 5,
       6, 3, 2, 5, 5, 3, 4, 2, 6, 3, 5, 2, 6, 8, 6, 6, 7, 6, 6, 1, 8, 6,
       4, 2, 4, 5, 5, 6, 5, 6, 8, 8, 4, 6, 4, 3, 3, 0, 4, 1, 3, 5, 3, 3,
       1, 6, 7, 0, 3, 7, 7, 5, 3, 2, 2, 3, 6, 3, 6, 3, 7, 3, 6, 5, 3, 8,
       4, 4, 6, 8, 6, 2, 4, 4, 5, 2, 4, 3, 2, 3, 2, 2, 8, 6, 3, 3, 6, 6,
       5, 2, 7, 4, 7, 7, 4, 6, 7, 4, 6, 7, 7, 6, 6, 6, 6, 6, 4, 4, 4, 4,
       1, 7, 6, 4, 7, 1, 4, 5, 7, 6, 5, 5, 7, 7, 4, 6, 1, 4, 7, 7, 1, 3,
       5, 1, 7, 1, 4, 5, 4, 6, 5, 8, 7, 7, 6, 7, 6, 4, 4, 7, 7, 7, 8, 6,
       1, 7, 1, 7, 6, 4, 5, 6, 5, 4, 7, 6, 5, 6, 1, 1, 6, 6, 4, 4, 4, 7,
       6, 4, 1, 4, 1, 6, 5, 4, 4, 6, 1, 6, 1, 5, 2,

In [6]:
# one_hot = np.eye(num_classes)[max_cats]
# print(one_hot[0])
# assert one_hot[0, 4] == 1
# one_hot

In [7]:
x, y = totals[:, 0], max_cats
x, y

(array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
         12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,
         23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,
         34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,
         45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,
         56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,  66.,
         67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,  77.,
         78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,
         89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,  99.,
        100., 101., 102., 103., 104., 105., 106., 107., 108., 109., 110.,
        111., 112., 113., 114., 115., 116., 117., 118., 119., 120., 121.,
        122., 123., 124., 125., 126., 127., 128., 129., 130., 131., 132.,
        133., 134., 135., 136., 137., 138., 139., 140., 141., 142., 143.,
        144., 145., 146., 147., 148., 

In [8]:
emotions = ['amazement', 'solemnity', 'tenderness', 'nostalgia', 'calmness', 'power', 'joyful_activation', 'tension', 'sadness']

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, Dataset
from torch.nn.utils.rnn import pad_sequence
import torchaudio

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [121]:
waveform, sample_rate = torchaudio.load('/datasets/memories/SongEmotionDataset/1.mp3')

In [122]:
waveform, sample_rate, waveform.shape

(tensor([[ 0.0000,  0.0000,  0.0000,  ..., -0.0007, -0.0024, -0.0029],
         [ 0.0000,  0.0000,  0.0000,  ..., -0.0011, -0.0029, -0.0033]]),
 44100,
 torch.Size([2, 2647296]))

In [123]:
mono = torch.mean(waveform, dim=0)
mono.shape, mono

(torch.Size([2647296]),
 tensor([ 0.0000,  0.0000,  0.0000,  ..., -0.0009, -0.0027, -0.0031]))

In [20]:
# settings
n_mfcc = 40
hidden_dim = 128
rnn_layers = 1

In [21]:
class SongEmotionDataset(Dataset):
    """
    Song Emotion Dataset. Uses librosa to process mp3 files.
    Takes first 20 seconds, and samples every 10 to get processed audio tensor.
    """

    def __init__(self, x, y, transform=None):
        """
        Args:
            mp3: list of paths to mp3 files
            labels: list of labels
        """
        self.x = x
        self.y = y
        self.cache = {}
        self.transform = torchaudio.transforms.MFCC(44100, n_mfcc)
        
    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        if index in self.cache:
            return self.cache[index]
        
        waveform, sample_rate = torchaudio.load(f'/datasets/memories/SongEmotionDataset/{int(x[index])}.mp3')
        mono_waveform = torch.mean(waveform, dim=0)
        mfcc = self.transform(mono_waveform).transpose(1, 0)[:5000]
        self.cache[index] = mfcc, torch.LongTensor([self.y[index]])
        return self.cache[index]

dataset = SongEmotionDataset(x, y)

In [22]:
traindata, testdata = random_split(dataset, [round(len(dataset)*.8), round(len(dataset)*.2)])

In [23]:
test_x, test_y = traindata[0]
print(test_x.shape, test_y.shape)

torch.Size([5000, 40]) torch.Size([1])


In [15]:
traindata[0]

(tensor([[-5.6110e+02,  5.3806e-06, -3.5935e-05,  ...,  6.2227e-05,
          -1.5855e-04, -1.3757e-04],
         [-5.6110e+02,  5.3806e-06, -3.5935e-05,  ...,  6.2227e-05,
          -1.5855e-04, -1.3757e-04],
         [-5.6110e+02,  5.3806e-06, -3.5935e-05,  ...,  6.2227e-05,
          -1.5855e-04, -1.3757e-04],
         ...,
         [-2.4912e+02,  8.0029e+01, -9.2529e+01,  ..., -1.8348e+01,
          -2.1697e+01, -6.8960e-01],
         [-2.4490e+02,  6.5710e+01, -7.6994e+01,  ..., -8.8169e+00,
          -1.6043e+01, -7.2640e+00],
         [-3.2050e+02,  4.1812e+01, -5.5972e+01,  ..., -5.8749e+00,
          -7.2861e+00, -6.6129e+00]]),
 tensor([6]))

In [16]:
test_y

tensor([6])

In [24]:
def pad_collate(batch):
    (xx, yy) = zip(*batch)
    xx_pad = pad_sequence(xx, batch_first=True, padding_value=0)
    yy_stack = torch.stack(yy).view(-1)
    return xx_pad, yy_stack

train_loader = DataLoader(traindata, batch_size=16, shuffle=True, num_workers=4, collate_fn=pad_collate)

In [178]:
test_x, test_y = next(iter(train_loader))

In [37]:
test_x.shape, test_y.shape

(torch.Size([5000, 40]), torch.Size([1]))

In [36]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.lstm = nn.LSTM(n_mfcc, hidden_dim, rnn_layers, batch_first=True)
        self.dense = dense = nn.Linear(hidden_dim, num_classes)
    def forward(self, x):
        output_seq, _ = self.lstm(inputs)
        # output_seq has shape [batch, sequence, hidden_dim]
        last_output = output_seq[:, -1]
        out = self.dense(last_output)
        return out

model = Net().to(device)

In [39]:
test_pred = model(test_x.to(device))
test_pred.shape

torch.Size([16, 9])

In [140]:
dense = nn.Linear(hidden_dim, num_classes).to(device)
last_output = test_pred[:, -1]
out = dense(last_output)

In [141]:
out.shape

torch.Size([1, 9])

In [143]:
test_y.shape

torch.Size([1, 1])

In [144]:
criterion(out, test_y.to(device).view(-1))

tensor(2.2591, device='cuda:0', grad_fn=<NllLossBackward>)

In [34]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [41]:
epochs = 5

for epoch in range(epochs):
    running_loss = 0.0
    for data in train_loader:
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        out = model(inputs)
        
        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        print(f'  - latest loss {loss.item()}')
    print(f'Epoch {epoch} - loss: {running_loss}')

print('Finished training')

  - latest loss 2.2128078937530518
  - latest loss 2.2544779777526855
  - latest loss 2.1909148693084717
  - latest loss 2.296206474304199
  - latest loss 2.200183153152466
  - latest loss 2.21799373626709
  - latest loss 2.220433235168457
  - latest loss 2.2295567989349365
  - latest loss 2.2066664695739746
  - latest loss 2.229936122894287
  - latest loss 2.1914634704589844
  - latest loss 2.1699812412261963
  - latest loss 2.179621934890747
  - latest loss 2.097020387649536
  - latest loss 2.2164359092712402
  - latest loss 2.1480345726013184
  - latest loss 2.169774055480957
  - latest loss 2.33172869682312
  - latest loss 2.2681100368499756
  - latest loss 2.327770709991455
Epoch 0 - loss: 44.35911774635315
  - latest loss 2.2703921794891357
  - latest loss 2.353577136993408
  - latest loss 2.231715202331543
  - latest loss 2.2401745319366455
  - latest loss 2.1168324947357178
  - latest loss 2.2503437995910645
  - latest loss 2.243340015411377
  - latest loss 2.162245035171509
  